In [19]:
import torch
from copy import deepcopy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer



In [20]:
from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset("sms_spam")
dataset = dataset.rename_column("sms", "text")
dataset = dataset['train']

processed_text = dataset['text']
display(type(processed_text))
processed_text = [s.replace('\n', '') for s in processed_text]

dataset = dataset.remove_columns(["text"])
display(dataset)
dataset = dataset.add_column("text", processed_text)
display(dataset)
dataset = dataset.shuffle(seed=42).select([i for i in list(range(5500))])


train_ds = Dataset.from_dict(dataset[0:4500])
validation_ds = Dataset.from_dict(dataset[4500:5000])
test_ds = Dataset.from_dict(dataset[5000:5500])

data = test_ds





Reusing dataset sms_spam (C:\Users\DELL\.cache\huggingface\datasets\sms_spam\plain_text\1.0.0\53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c)
100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 991.56it/s]


list

Dataset({
    features: ['label'],
    num_rows: 5574
})

Dataset({
    features: ['label', 'text'],
    num_rows: 5574
})

Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\sms_spam\plain_text\1.0.0\53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c\cache-1dabca71be75cbef.arrow


In [21]:
display(data)
display(data[0:5])

Dataset({
    features: ['label', 'text'],
    num_rows: 500
})

{'label': [0, 0, 0, 0, 1],
 'text': ['Yeah so basically any time next week you can get away from your mom &amp; get up before 3',
  "Haha... can... But i'm having dinner with my cousin...",
  'How is your schedule next week? I am out of town this weekend.',
  'Alright i have a new goal now',
  'Check Out Choose Your Babe Videos @ sms.shsex.netUN fgkslpoPW fgkslpo']}

In [22]:
import textattack
dataset = textattack.datasets.HuggingFaceDataset(data)
display(dataset)

In [23]:
#attack the original model using TextFooler
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu") 
model_path = r'C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\models\CheckList\run3'
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)



In [24]:
from textattack.attack_recipes.pwws_ren_2019 import PWWSRen2019
attack = PWWSRen2019.build(model_wrapper)
attack.cuda_()
display(attack)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)

In [25]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

attack_args = AttackArgs(num_examples=-1, random_seed = 765,
                         log_to_csv = r'C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\resullts\CheckList\run3\pwws_attack_results.csv', 
                         csv_coloring_style = 'file')

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()


textattack: Logging to CSV at path C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\resullts\CheckList\run3\pwws_attack_results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



  0%|▏                                                                                 | 1/500 [00:01<13:41,  1.65s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   0%|                            | 1/500 [00:01<13:46,  1.66s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:   0%|                            | 2/500 [00:01<07:16,  1.14it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

Yeah so basically any time next week you can get away from your mom &amp; get up before 3


--------------------------------------------- Result 2 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

Haha... can... But i'm having dinner with my cousin...




[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:   1%|▏                           | 3/500 [00:02<05:33,  1.49it/s]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 0 / 3 / 0 / 3:   1%|▏                           | 3/500 [00:02<05:35,  1.48it/s]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)


...

--------------------------------------------- Result 497 ---------------------------------------------
[[0 (99%)]] --> [[1 (82%)]]

[[Book]] which lesson? then you msg me... I will [[call]] up after work or sth... I'm going to get specs. My membership is PX3748

[[playscript]] which lesson? then you msg me... I will [[phone]] up after work or sth... I'm going to get specs. My membership is PX3748




C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 55 / 440 / 3 / 498: 100%|████████████████████▉| 499/500 [06:33<00:00,  1.27it/s]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)


--------------------------------------------- Result 498 ---------------------------------------------
[[1 (100%)]] --> [[0 (56%)]]

U have a secret [[admirer]] who is [[looking]] [[2]] [[make]] [[contact]] with U-find out who they R*reveal who [[thinks]] UR so special-call on 09058094599

U have a secret [[adorer]] who is [[seem]] [[II]] [[constitute]] [[adjoin]] with U-find out who they R*reveal who [[suppose]] UR so special-call on 09058094599


--------------------------------------------- Result 499 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

You'd like that wouldn't you? Jerk!




[Succeeded / Failed / Skipped / Total] 55 / 441 / 3 / 499: 100%|████████████████████▉| 499/500 [06:33<00:00,  1.27it/s]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 55 / 442 / 3 / 500: 100%|█████████████████████| 500/500 [06:35<00:00,  1.27it/s]

--------------------------------------------- Result 500 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

Not a lot has happened here. Feels very quiet. Beth is at her aunts and charlie is working lots. Just me and helen in at the mo. How have you been? 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 55     |
| Number of failed attacks:     | 442    |
| Number of skipped attacks:    | 3      |
| Original accuracy:            | 99.4%  |
| Accuracy under attack:        | 88.4%  |
| Attack success rate:          | 11.07% |
| Average perturbed word %:     | 28.7%  |
| Average num. words per input: | 15.39  |
| Avg num queries:              | 185.22 |
+-------------------------------+--------+


In [26]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples

logger = CSVLogger(color_method='html')

for result in attack_results:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))
results_html = logger.df[['original_text', 'perturbed_text']].to_html(escape=False)
path = r'C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\resullts\CheckList\run3\pwws_attack_results.html'
text_file = open(path, "w")
text_file.write(results_html.encode("gbk", 'ignore').decode("gbk", "ignore"))
text_file.close()
logger.flush()

textattack: Logging to CSV at path results.csv
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated 

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

...

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

,original_text,perturbed_text
0,Yeah so basically any time next week you can get away from your mom & get up before 3,Yeah so basically any meter future week you can arrive aside from your mum &A; puzzle up before triplet
1,Haha... can... But i'm having dinner with my cousin...,Haha... can... just i'm having dinner with my cousin...
2,How is your schedule next week? I am out of town this weekend.,How is your agenda future workweek? single am out of townsfolk this weekend.
3,Alright i have a new goal now,okay i have a new destination now
4,Check Out Choose Your Babe Videos @ sms.shsex.netUN fgkslpoPW fgkslpo,Check Out Choose Your Babe Videos @ sms.shsex.netUN fgkslpoPW fgkslpo
5,Send a logo 2 ur lover - 2 names joined by a heart. Txt LOVE NAME1 NAME2 MOBNO eg LOVE ADAM EVE 07123456789 to 87077 Yahoo! POBox36504W45WQ TxtNO 4 no ads 150p.,broadcast a logo II ur fan - 2 gens coupled by a kernel. Txt jazz NAME1 NAME2 MOBNO eg hump XTC even 07123456789 to 87077 bumpkin! POBox36504W45WQ TxtNO IV no ad 150p.
6,Its a site to simulate the test. It just gives you very tough questions to test your readiness.,IT a website to simulate the tryout. It just gives you very rugged questions to quiz your set.
7,"Sorry, I'll call later","good-for-nothing, I'll scream after"
8,Allo! We have braved the buses and taken on the trains and triumphed. I mean we‘re in b‘ham. Have a jolly good rest of week,Allo! We have braved the double-decker and taken on the geartrain and triumphed. 1 hateful we‘re in boron‘overplay. feature a josh expert ease of week
9,Yeah. I got a list with only u and Joanna if I'm feeling really anti social,Yeah. iodine receive a listing with only uranium and Joanna if I'm feeling genuinely anti social


In [27]:
import textattack

attack_log_manager = textattack.loggers.AttackLogManager()


for result in attack_results:
    attack_log_manager.log_result(result)

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()

attack_log_manager.flush()


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 55     |
| Number of failed attacks:     | 442    |
| Number of skipped attacks:    | 3      |
| Original accuracy:            | 99.4%  |
| Accuracy under attack:        | 88.4%  |
| Attack success rate:          | 11.07% |
| Average perturbed word %:     | 28.7%  |
| Average num. words per input: | 15.39  |
| Avg num queries:              | 185.22 |
+-------------------------------+--------+